In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Open the jsonfile and display the first level of keys
# open API results WITH json module
import json
with open('Data/example_yelp_results.json') as f:   #adjust for your path
    json_file = json.load(f)
    
## What type is top-level of json?
type(json_file)

dict

In [3]:
# what are the keys?
json_file.keys()

dict_keys(['businesses', 'total', 'region'])

In [4]:
# what type is region?
type(json_file['region'])

dict

In [5]:
# what keys are in region?
json_file['region'].keys()

dict_keys(['center'])

In [6]:
# what type is center?
type(json_file['region']['center'])

dict

In [7]:
# what are the keys in center?
json_file['region']['center'].keys()

dict_keys(['longitude', 'latitude'])

In [8]:
# Visualize the whole center dict, since lat and long are likely single values
json_file['region']['center']

{'longitude': -76.607666015625, 'latitude': 39.29617825404647}

In [9]:
# what is stored under the "total" key?
type(json_file['total'])

int

In [10]:
## what is the value?
json_file['total']

435

In [11]:
# what is in the businesses key?
type(json_file['businesses'])

list

In [12]:
# how long is businesses?
len(json_file['businesses'])

20

In [13]:
# what does the first entry of business look like?
json_file['businesses'][0]

{'id': 'D9A33FM394q99o4QtK5YwA',
 'alias': 'faidleys-seafood-baltimore-3',
 'name': 'Faidleys Seafood',
 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/OTjVDCVS7pGopH6GZcfjBA/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/faidleys-seafood-baltimore-3?adjust_creative=KJtcufKUS887p24u6rjVIQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=KJtcufKUS887p24u6rjVIQ',
 'review_count': 1184,
 'categories': [{'alias': 'seafood', 'title': 'Seafood'},
  {'alias': 'beerbar', 'title': 'Beer Bar'}],
 'rating': 4.0,
 'coordinates': {'latitude': 39.291696, 'longitude': -76.62224},
 'transactions': ['delivery'],
 'price': '$$',
 'location': {'address1': '203 N Paca St',
  'address2': '',
  'address3': '',
  'city': 'Baltimore',
  'zip_code': '21201',
  'country': 'US',
  'state': 'MD',
  'display_address': ['203 N Paca St', 'Baltimore, MD 21201']},
 'phone': '+14107274898',
 'display_phone': '(410) 727-4898',
 'distance': 1349.560720156645}

In [14]:
## what are the keys of the first entry in businesses?
json_file['businesses'][0].keys()

dict_keys(['id', 'alias', 'name', 'image_url', 'is_closed', 'url', 'review_count', 'categories', 'rating', 'coordinates', 'transactions', 'price', 'location', 'phone', 'display_phone', 'distance'])

In [16]:
df_businesses = pd.DataFrame(json_file['businesses'])
df_businesses.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,D9A33FM394q99o4QtK5YwA,faidleys-seafood-baltimore-3,Faidleys Seafood,https://s3-media3.fl.yelpcdn.com/bphoto/OTjVDC...,False,https://www.yelp.com/biz/faidleys-seafood-balt...,1184,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,"{'latitude': 39.291696, 'longitude': -76.62224}",[delivery],$$,"{'address1': '203 N Paca St', 'address2': '', ...",+14107274898,(410) 727-4898,1349.560720
1,u65W69AhbjUlvJJBkEhGNQ,miss-shirleys-cafe-baltimore-9,Miss Shirley's Cafe,https://s3-media4.fl.yelpcdn.com/bphoto/9FsOyV...,False,https://www.yelp.com/biz/miss-shirleys-cafe-ba...,2919,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.0,"{'latitude': 39.2870995, 'longitude': -76.6053...","[delivery, pickup]",$$,"{'address1': '750 E Pratt St', 'address2': '',...",+14105285373,(410) 528-5373,1028.736468
2,ieS_5zqxDHcWMCm8BKUYbg,thames-street-oyster-house-baltimore,Thames Street Oyster House,https://s3-media1.fl.yelpcdn.com/bphoto/9hGjo5...,False,https://www.yelp.com/biz/thames-street-oyster-...,2729,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.5,"{'latitude': 39.28214, 'longitude': -76.59162}",[delivery],$$$,"{'address1': '1728 Thames St', 'address2': '',...",+14434497726,(443) 449-7726,2090.712792
3,6am8TZAFnvND52MOz-Yctg,mamas-on-the-half-shell-baltimore,Mama's On The Half Shell,https://s3-media2.fl.yelpcdn.com/bphoto/HWY8OF...,False,https://www.yelp.com/biz/mamas-on-the-half-she...,1279,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"{'latitude': 39.27986, 'longitude': -76.5752399}","[delivery, pickup]",$$,"{'address1': '2901 Odonnell St', 'address2': '...",+14102763160,(410) 276-3160,3328.825798
4,ISn7WyGQaFpsVSRSh0NSqg,sal-and-sons-baltimore,Sal and Sons,https://s3-media3.fl.yelpcdn.com/bphoto/LmVL4j...,False,https://www.yelp.com/biz/sal-and-sons-baltimor...,153,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.5,"{'latitude': 39.284, 'longitude': -76.59337}",[delivery],$,"{'address1': '1640 Aliceanna St', 'address2': ...",+14106751466,(410) 675-1466,1817.406978


In [17]:
df_businesses[['categories','coordinates','location']].head(3)

,categories,coordinates,location
0,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...","{'latitude': 39.291696, 'longitude': -76.62224}","{'address1': '203 N Paca St', 'address2': '', ..."
1,"[{'alias': 'breakfast_brunch', 'title': 'Break...","{'latitude': 39.2870995, 'longitude': -76.6053...","{'address1': '750 E Pratt St', 'address2': '',..."
2,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...","{'latitude': 39.28214, 'longitude': -76.59162}","{'address1': '1728 Thames St', 'address2': '',..."


In [18]:
import json
with open('/Users/abigailbojorquez/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['Client-id', 'API-key'])

In [20]:
# import the YelpAPI Class
from yelpapi import YelpAPI
# Create an instance with your key
yelp_api = YelpAPI(login['API-key'], timeout_s=5.0)
yelp_api

In [21]:
help(yelp_api.search_query)

Help on method search_query in module yelpapi.yelpapi:

search_query(**kwargs) method of yelpapi.yelpapi.YelpAPI instance
    Query the Yelp Search API.
    
    documentation: https://www.yelp.com/developers/documentation/v3/business_search
    
    required parameters:
        * one of either:
            * location - text specifying a location to search for
            * latitude and longitude



In [22]:
# use our yelp_api variable's search_query method to perform our API call
search_results = yelp_api.search_query(location='NY, NY',
                                       term='Pizza')
print(type(search_results))
search_results.keys()

<class 'dict'>


dict_keys(['businesses', 'total', 'region'])

In [24]:
search_results['total']



16000

In [25]:
biz = pd.DataFrame(search_results['businesses'])
biz.head(2) 

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,zj8Lq1T8KIC5zwFief15jg,prince-street-pizza-new-york-2,Prince Street Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/I4gm7i...,False,https://www.yelp.com/biz/prince-street-pizza-n...,4702,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.72308755605564, 'longitude': -...","[pickup, delivery]",$,"{'address1': '27 Prince St', 'address2': None,...",+12129664100,(212) 966-4100,2209.311618
1,ysqgdbSrezXgVwER2kQWKA,julianas-brooklyn-3,Juliana's,https://s3-media2.fl.yelpcdn.com/bphoto/NVoLFl...,False,https://www.yelp.com/biz/julianas-brooklyn-3?a...,2599,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.70274718768062, 'longitude': -...",[delivery],$$,"{'address1': '19 Old Fulton St', 'address2': '...",+17185966700,(718) 596-6700,1289.857286


In [27]:
# add offset to our original api call
search_results = yelp_api.search_query(location='NY, NY',
                                       term='Pizza',
                                       offset = 20)

In [28]:
biz20 = pd.DataFrame(search_results['businesses'])
biz20.head(2)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,6ECcDYsRgmfv0eKXk-c11Q,la-nonna-krispy-krust-pizza-brooklyn,La Nonna Krispy Krust Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/fVNfYk...,False,https://www.yelp.com/biz/la-nonna-krispy-krust...,74,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.717953, 'longitude': -73.9637027}","[pickup, delivery]",$$,"{'address1': '175 Kent Ave', 'address2': None,...",+13473842100,(347) 384-2100,1786.793379
1,CiRj1B-gK5tRfbMkq3SBog,robertas-new-york-3,Roberta's,https://s3-media3.fl.yelpcdn.com/bphoto/__bI5t...,False,https://www.yelp.com/biz/robertas-new-york-3?a...,4,"[{'alias': 'pizza', 'title': 'Pizza'}]",5.0,"{'latitude': 40.71995520720623, 'longitude': -...","[pickup, delivery]",NaN,"{'address1': '196 Stanton St', 'address2': '',...",,,1479.203775


In [29]:
## concatenate the previous results and new results. 
businesses = pd.concat([biz, biz20],
                      ignore_index=True)
display(businesses.head(3), businesses.tail(3))

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,zj8Lq1T8KIC5zwFief15jg,prince-street-pizza-new-york-2,Prince Street Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/I4gm7i...,False,https://www.yelp.com/biz/prince-street-pizza-n...,4702,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.72308755605564, 'longitude': -...","[pickup, delivery]",$,"{'address1': '27 Prince St', 'address2': None,...",+12129664100,(212) 966-4100,2209.311618
1,ysqgdbSrezXgVwER2kQWKA,julianas-brooklyn-3,Juliana's,https://s3-media2.fl.yelpcdn.com/bphoto/NVoLFl...,False,https://www.yelp.com/biz/julianas-brooklyn-3?a...,2599,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.70274718768062, 'longitude': -...",[delivery],$$,"{'address1': '19 Old Fulton St', 'address2': '...",+17185966700,(718) 596-6700,1289.857286
2,v1DHGRNCH9247WLYoaoA9A,l-industrie-pizzeria-brooklyn,L'industrie Pizzeria,https://s3-media4.fl.yelpcdn.com/bphoto/NT9kSP...,False,https://www.yelp.com/biz/l-industrie-pizzeria-...,854,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.71162, 'longitude': -73.95783}",[delivery],$,"{'address1': '254 S 2nd St', 'address2': '', '...",+17185990002,(718) 599-0002,1868.295512


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
37,aP1Nwq1-7j0Y7hr7oavY2A,upside-pizza-new-york-2,Upside Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/RWcIDI...,False,https://www.yelp.com/biz/upside-pizza-new-york...,109,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.722115, 'longitude': -73.996407}","[pickup, delivery]",NaN,"{'address1': '51 Spring St', 'address2': '', '...",,,2221.489967
38,2FgUndzCT_jJATbmrBDVBg,paulie-gees-slice-shop-brooklyn,Paulie Gee's Slice Shop,https://s3-media4.fl.yelpcdn.com/bphoto/So5FTa...,False,https://www.yelp.com/biz/paulie-gees-slice-sho...,307,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 40.72836, 'longitude': -73.9572}",[delivery],$,"{'address1': '110 Franklin St', 'address2': ''...",,,3016.076927
39,affWh2WFSMayL04l9lr47g,roberta-s-brooklyn,Roberta’s,https://s3-media1.fl.yelpcdn.com/bphoto/rdK-so...,False,https://www.yelp.com/biz/roberta-s-brooklyn?ad...,11,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.71613781459604, 'longitude': -...",[],NaN,"{'address1': '6 Grand St', 'address2': '', 'ad...",,,1471.462333
